# Transfer learning gender classifier by using dlib face feature extractor

In [2]:
import os
import timeit
import cv2
from skimage import io as io
import face_recognition as fr
import numpy as np
import pickle
from tqdm import tqdm
from sklearn import datasets, svm, metrics
from sklearn.model_selection import GridSearchCV
import random
from matplotlib import pyplot as plt
import pandas as pd

%matplotlib inline

In [43]:
with open('celeba_smiling_data.pkl', 'rb') as f:
    gender_dataset_raw = pickle.load(f)
random.shuffle(gender_dataset_raw)

In [44]:
len(gender_dataset_raw)

4865

In [45]:
embedding_list_train = list()
gender_label_list_train = list()

embedding_list_test = list()
gender_label_list_test = list()

for emb, label in gender_dataset_raw[0:4000]:
    embedding_list_train.append(emb)
    gender_label_list_train.append(label)

for emb, label in gender_dataset_raw[4000:]:
    embedding_list_test.append(emb)
    gender_label_list_test.append(label)
    
print('length of embedding train list: {}'.format(len(embedding_list_train)))
print('lenght of label train list: {}'.format(len(gender_label_list_train)))
print('length of embedding test list: {}'.format(len(embedding_list_test)))
print('lenght of label test list: {}'.format(len(gender_label_list_test)))

length of embedding train list: 4000
lenght of label train list: 4000
length of embedding test list: 865
lenght of label test list: 865


In [ ]:
param_grid = {'C': [1e1, 1e2],
              'gamma': [10, 100], 
              'decision_function_shape':["ovo"],
                "degree":[3,4]}
classifier = GridSearchCV(svm.SVC(kernel='rbf', class_weight="balanced", verbose=True),
                   param_grid, cv=5)
# classifier = svm.SVC(gamma='auto', kernel='rbf', C=20, degree=3, class_weight="balanced", 
#                     verbose=True)
classifier.fit(embedding_list_train, gender_label_list_train)

expected = gender_label_list_test
predicted = classifier.predict(embedding_list_test)

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [ ]:
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Classification report for classifier GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'degree': [3, 4], 'C': [10.0, 100.0], 'decision_function_shape': ['ovo'], 'gamma': [10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0):
              precision    recall  f1-score   support

          -1       0.68      0.73      0.70       446
           1       0.69      0.63      0.66       419

   micro avg       0.68      0.68      0.68       865
   macro avg       0.68      0.68      0.68       865
weighted avg       0.68      0.68      0.68       865


Confusion matrix:
[[326 120]
 [155 264]]

In [ ]:
classifier.best_estimator_

# Test

In [ ]:
test_img_path = "/home/ubuntu/ihandy_seg/data/img_align_celeba"
test_imgs = [f for f in os.listdir(test_img_path) if not f.startswith('.')]
table = pd.read_csv("/home/ubuntu/ihandy_seg/data/list_attr_celeba.csv")
img_label = table.set_index('image_id').to_dict()["Male"]

In [ ]:
correct_count = 0
test_count = 0
for i, img_file in enumerate(test_imgs[:100]):
    img = io.imread(os.path.join(test_img_path, img_file))
    try:
        img_enc = fr.face_encodings(img)[0]
    except:
        print("None Face in this image:", img_file)
        continue
    test_count += 1
    img_predict = classifier.predict([img_enc])
    if img_predict[0] == 1 and img_label[img_file] == 1:
        correct_count += 1
    if img_predict[0] == -1 and img_label[img_file] == -1:
        correct_count += 1
    plt.title("The prediced gender: {}".format(img_predict))
    plt.title(img_predict)
    plt.imshow(img)
    plt.show()
print("accuracy:", correct_count / test_count)


In [ ]:
img1 = io.imread('test1.jpg')
img2 = io.imread('test2.jpg')

img1_enc = fr.face_encodings(img1)[0]
img2_enc = fr.face_encodings(img2)[0]

img1_predict = classifier.predict([img1_enc])
img2_predict = classifier.predict([img2_enc])

plt.title('The predicted gender: {}'.format(img1_predict))
plt.imshow(img1)
plt.show()

plt.title('The predicted gender: {}'.format(img2_predict))
plt.imshow(img2)
plt.show()